In [1]:
df_filtered.head()

NameError: ignored

In [166]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [167]:
!cp '/content/drive/My Drive/HSE_LIB/CompLingv/all_train.csv' 'all_train.csv'
!cp '/content/drive/My Drive/HSE_LIB/CompLingv/processed_data.csv' 'processed_data.csv'

In [4]:
import pandas as pd
import numpy as np
import os
import json 
import re
import csv
import string
import nltk
import ast
import re
import warnings
from tqdm.notebook import tqdm as tqdm
from torch import nn
import torch
from sklearn.model_selection import train_test_split

In [5]:
# disabling warnings about max length of tokens
import logging
import re
def set_global_logging_level(level=logging.ERROR, prefices=[""]):
    prefix_re = re.compile(fr'^(?:{ "|".join(prefices) })')
    for name in logging.root.manager.loggerDict:
        if re.match(prefix_re, name):
            logging.getLogger(name).setLevel(level)
set_global_logging_level(logging.ERROR, ["transformers", "nlp", "torch", "tensorflow", "tensorboard", "wandb"])

In [6]:
RESOURCES_PATH = os.curdir
DRIVE_PATH = './'
BATCH_SIZE = 12
N_EPOCH = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
# df = pd.read_csv(os.path.join(RESOURCES_PATH, 'all_train.csv'))
# df = df.sample(frac=1.).reset_index(drop=True)
# df_train, df_test = df[:25000], df[25000:]
# # texts_train, texts_test, y_train, y_test = \
# # train_test_split(df['Text'].values, df['FinalScore1'].values, test_size=0.2, random_state=1000)

In [61]:
df = pd.read_csv(os.path.join(RESOURCES_PATH, 'processed_data.csv'))
df_filtered = df.drop(columns=["Score", "Category", "tokens", "pos", "lemmas", "is post"])
df_filtered["classical_features"] = [np.array([x1, y1, int(x2), y2, x3, y3, x4, y4, x5, y5]) for x1, y1, x2, y2, x3, y3, x4, y4, x5, y5 in zip(df_filtered["nouns percent"], df_filtered["adjectives percent"],
                                                                       df_filtered["upper case"], df_filtered["verbs percent"],
                                                                       df_filtered["emotional verbs"], df_filtered["obscene words"],
                                                                       df_filtered["average word length"], df_filtered["exclamation mark count"],
                                                                       df_filtered["question mark count"], df_filtered["average word sentiment"])]
df_filtered  = df_filtered.filter(["Text", "classical_features", "FinalScore1"])
df = df_filtered.sample(frac=1.).reset_index(drop=True)
# df_train, df_test = df[:9000], df[9000:10000]
df_train, df_test = df[:25000], df[25000:]

In [74]:
!cp 'test.json' '/content/drive/My Drive/HSE_LIB/CompLingv/test7.json' 

In [72]:
df_train[['Text', 'FinalScore1']].to_csv('train.csv')
df_test[['Text', 'FinalScore1']].to_csv('test.csv')
df_train.to_json('train.json',orient='records', force_ascii=False)
df_test.to_json('test.json', orient='records', force_ascii=False)

df_train[['Text', 'FinalScore1']][:100].to_csv('train_smol.csv')
df_test[['Text', 'FinalScore1']][:100].to_csv('test_smol.csv')

In [75]:
with open('test.json', 'w') as f:
    for _, r in df.iterrows():
        r.to_json(f,force_ascii=False); f.write('\n')
with open('train.json', 'w') as f:
    for _, r in df.iterrows():
        r.to_json(f,force_ascii=False); f.write('\n')

In [55]:
# !sed '$ s/.$//' test.json > test.json

In [56]:
# !tail -c +2 test.json > test.json

In [23]:
# import json

# with open('test.json') as json_file:
#     data = json.load(json_file)

In [7]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', tokenizer_language = 'ru_core_news_lg')

CLASSIC = data.Field(use_vocab=False)

LABEL = data.LabelField()


# fields=[
#   (None, None),
#   ('Text', TEXT),
#   ('classical_features', CLASSIC),
#   ('FinalScore1', LABEL)
# ]
fields={
  'Text': ('Text', TEXT),
  'classical_features': ('classical_features', CLASSIC),
  'FinalScore1': ('FinalScore1', LABEL)
}

# train_data, test_data = data.TabularDataset.splits(TEXT, LABEL, fine_grained=False)
train, test = data.TabularDataset.splits(path='./', train='train.json', test='test.json', format='json', fields=fields)#, skip_header=True)
# train_smol, test_smol = data.TabularDataset.splits(path='./', train='train_smol.csv', test='test_smol.csv', format='csv', fields=fields, skip_header=True)

train_data, valid_data = train.split(random_state = random.seed(SEED))

ModuleNotFoundError: No module named 'torch'

In [ ]:
# data.Field

In [27]:
# df_filtered['classical_features']

0        [0.3898305084745763, 0.15254237288135594, 1.0,...
1        [0.2573099415204678, 0.1286549707602339, 0.0, ...
2        [0.4, 0.14545454545454545, 0.0, 0.090909090909...
3        [0.453551912568306, 0.08743169398907104, 1.0, ...
4        [0.175, 0.075, 0.0, 0.1625, 0.0, 0.0, 5.0875, ...
                               ...                        
29437    [0.2080536912751678, 0.026845637583892617, 1.0...
29438    [0.19672131147540986, 0.08196721311475409, 0.0...
29439    [0.2598039215686275, 0.06862745098039216, 1.0,...
29440    [0.2079646017699115, 0.04424778761061947, 0.0,...
29441    [0.2972972972972973, 0.0945945945945946, 0.0, ...
Name: classical_features, Length: 29442, dtype: object

In [78]:
train_data[0].classical_features

[0.2400990099,
 0.0643564356,
 1.0,
 0.1138613861,
 0.0,
 2.0,
 4.2846534653,
 28.0,
 14.0,
 -0.0325581395]

In [ ]:
vars(train_data[-1])

In [87]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.300d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)


In [85]:
print(LABEL.vocab.stoi)

defaultdict(None, {0: 0, -1: 1, 1: 2})
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f8e6ca916d0>>, {'<unk>': 0, '<pad>': 1, 0.0: 2, 1.0: 3, 2.0: 4, 3.0: 5, 4.0: 6, 5.0: 7, 6.0: 8, 7.0: 9, 0.1111111111: 10, 0.125: 11, 0.1: 12, 0.0909090909: 13, 8.0: 14, 0.0833333333: 15, 0.1428571429: 16, 0.0769230769: 17, 0.25: 18, 0.1176470588: 19, 0.3333333333: 20, 9.0: 21, 0.0714285714: 22, 0.0952380952: 23, 0.1333333333: 24, 0.1052631579: 25, 0.2: 26, 0.0666666667: 27, 0.1666666667: 28, 10.0: 29, 0.0625: 30, 0.0588235294: 31, 0.12: 32, 0.08: 33, 0.0967741935: 34, 0.0555555556: 35, 0.0869565217: 36, 0.1034482759: 37, 0.09375: 38, 0.1153846154: 39, 0.0526315789: 40, 0.0882352941: 41, 11.0: 42, 0.2857142857: 43, 0.0476190476: 44, 0.0857142857: 45, 0.1071428571: 46, 0.0434782609: 47, 0.0454545455: 48, 0.1538461538: 49, 0.1063829787: 50, 0.1304347826: 51, 0.1363636364: 52, 0.1081081081: 53, -0.05: 54, 0.2222222222: 55, 0.1290322581: 56, -0.0555555556: 57, 0.0

In [82]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test), 
    batch_size = BATCH_SIZE,
    sort=False,
    device = device)

In [88]:
for x in train_iterator:
  print(x)
  break


[torchtext.legacy.data.batch.Batch of size 128]
	[.Text]:[torch.cuda.LongTensor of size 460x128 (GPU 0)]
	[.classical_features]:[torch.cuda.LongTensor of size 10x128 (GPU 0)]
	[.FinalScore1]:[torch.cuda.LongTensor of size 128 (GPU 0)]


In [91]:
x.classical_features.shape

torch.Size([10, 128])

In [8]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, classical_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx, hidden_linear_dim=64):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # self.lstm = nn.LSTM(embedding_dim, embedding_dim // 2)
        # self.lstm_drop = nn.Dropout(1 - dropout)
        
        conv_nets = [nn.Conv2d(in_channels = 1, out_channels = n_filters, kernel_size = (fs, embedding_dim)) for fs in filter_sizes]
        self.convs = nn.ModuleList(conv_nets)
        
        # self.fc1 = nn.Linear(len(filter_sizes) * n_filters, len(filter_sizes) * n_filters // 2)
        self.fc_conv_model = nn.Linear(len(filter_sizes) * n_filters, hidden_linear_dim)
        self.fc_classical_features = nn.Linear(classical_dim, hidden_linear_dim)
        print
        self.fc = nn.Linear(hidden_linear_dim * 2, output_dim)
        print('class dim', classical_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, clasical_features):
        
        text = text.permute(1, 0)

        embedded = self.embedding(text)
                

        # embedded, _ = self.lstm(embedded)
        # embedded = self.lstm_drop(embedded)
        embedded = F.relu(embedded.unsqueeze(1))

        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        cat = F.relu(self.fc_conv_model(cat))
        print('kek', clasical_features.shape, self.fc_classical_features.weight.shape)
        cat_class = F.relu(self.fc_classical_features(clasical_features))
        cat = torch.cat((cat, cat_class))
            
        return self.fc(cat)

ModuleNotFoundError: No module named 'torch'

In [174]:
model.embedding

Embedding(25002, 100)

In [1]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 80
FILTER_SIZES = [4,3,2]
OUTPUT_DIM = len(LABEL.vocab)
CLASSICAL_DIM = 10
DROPOUT = 0.75
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, CLASSICAL_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)


NameError: name 'TEXT' is not defined

In [ ]:
m

In [154]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.1077,  0.1105,  0.5981,  ..., -0.8316,  0.4529,  0.0826],
        ...,
        [ 0.4381,  1.6938,  0.3594,  ..., -1.5819,  1.2080,  2.2264],
        [-0.1189,  0.3157,  0.9354,  ...,  0.3449, -1.1498, -1.1058],
        [-1.9875,  0.4721, -0.2410,  ..., -0.5068, -0.5407,  0.4570]])

In [156]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(),lr=1e-3)
criterion = nn.CrossEntropyLoss()
# CUDA_LAUNCH_BLOCKING=1
model = model.to(device)
criterion = criterion.to(device)

In [3]:
from sklearn.metrics import f1_score
def categorical_accuracy(preds, y):
    # top_pred = preds.argmax(1, keepdim = True)
    # correct = top_pred.eq(y.view_as(top_pred)).sum()
    # acc = correct.float() / y.shape[0]
    # return f1_score(y.detach().cpu().numpy(), top_pred.detach().cpu().numpy(), average='macro'), acc
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return f1_score(y.detach().cpu().numpy(), top_pred.detach().cpu().numpy(), average='macro'), acc

ModuleNotFoundError: No module named 'sklearn'

In [4]:
def train_f(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_f1 = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        predictions = model(batch.Text, batch.classical_features).squeeze(1)       
        loss = criterion(predictions, batch.FinalScore1)     
        f1, acc = categorical_accuracy(predictions, batch.FinalScore1)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_f1 += f1
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_f1 / len(iterator)

In [5]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    epoch_f1 = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.Text, batch.classical_features).squeeze(1) 
            loss = criterion(predictions, batch.FinalScore1) 
            f1, acc = categorical_accuracy(predictions, batch.FinalScore1)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_f1 += f1    
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_f1 / len(iterator)

In [10]:
import time

def epoch_time(start, end):
    elapsed_time = end - start
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [9]:
N_EPOCHS = 50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):  
    start = time.time()  
    train_loss, train_acc, train_f1 = train_f(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, valid_f1 = evaluate(model, test_iterator, criterion)      
    end = time.time()     
    epoch_mins, epoch_secs = epoch_time(start, end)     
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'cnn-model.pt') 
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f} | Train f1: {train_f1}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f} | Val. f1: {valid_f1}%')

NameError: name 'model' is not defined

In [ ]:
  !cp '/content/cnn-model.pt' '/content/drive/My Drive/HSE_LIB/CompLingv/cnn-model-2.pt'

In [ ]:
!pip install -U spacy==3.0.5

In [21]:
!python -m spacy download ru_core_news_lg

2021-03-31 12:05:54.857083: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 515.3MB 33kB/s 
     |████████████████████████████████| 61kB 4.0MB/s 
     |████████████████████████████████| 8.2MB 7.8MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [25]:
spacy.util.set_data_path('/content/')

In [1]:
import spacy
nlp = spacy.load("ru_core_news_lg")

KeyboardInterrupt: ignored